In [1]:
IS_GPU = False
# Импорт нужных библиотек
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import time
from scipy.optimize import minimize
from neighbors import Neighborhoods

from indices import MainDataset
from dnn_utils import preprocess_floor
from metric import metrics_stat, deviation_metric
from catboost import CatBoostRegressor
from catboost import Pool

def reset_tensorflow_session():
    tf.keras.backend.clear_session()
    tf.random.set_seed(41)
    np.random.seed(41)


THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1

In [2]:

# Категориальные данные
CATEGORICAL_FEATURES_COLUMNS = ['region', 'city', 'realty_type', 'floor', 'osm_city_nearest_name', 'street']
# Численные данные
NUM_FEATURES_COLUMNS = ['lat', 'lng', 'osm_amenity_points_in_0.001',
                        'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
                        'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
                        'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
                        'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
                        'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
                        'osm_catering_points_in_0.01', 'osm_city_closest_dist',
                        'osm_city_nearest_population',
                        'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
                        'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
                        'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
                        'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
                        'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
                        'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
                        'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
                        'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
                        'osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075',
                        'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
                        'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
                        'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
                        'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
                        'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
                        'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
                        'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
                        'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
                        'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
                        'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
                        'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
                        'osm_transport_stop_points_in_0.0075',
                        'osm_transport_stop_points_in_0.01',
                        'reform_count_of_houses_1000', 'reform_count_of_houses_500',
                        'reform_house_population_1000', 'reform_house_population_500',
                        'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
                        'reform_mean_year_building_1000', 'reform_mean_year_building_500', 'total_square',
                        "neighbor_dist", "neighbor_total_price", "neighbor_square_price", "neighbor10_dist",
                        "has_basement", "floor_count"

                        ]
# Таргет
TARGET_COLUMNS = ['per_square_meter_price']

In [3]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
train = train[train.price_type == 1].reset_index(drop=True)
train['is_train'] = 1
test['is_train'] = 0
dataset = pd.concat([train, test]).reset_index(drop=True)


In [4]:
train_dataset_index = MainDataset("dataset/train.csv")
test_dataset_index = MainDataset("dataset/test.csv", need_index=False)
neighborhoods = Neighborhoods(train_dataset_index.index)


In [5]:
dataset["neighbor_dist"] = -999
dataset["neighbor_total_price"] = -999
dataset["neighbor_square_price"] = -999
dataset["neighbor10_dist"] = -999
for d in [test_dataset_index, train_dataset_index]:
    for i, o in enumerate(d.all_objects):
        if o.row["price_type"] != 1:
            continue
        neighbor = neighborhoods.get_haversine_closest(o, 12)
        neighbor1 = neighborhoods.get_haversine_closest(o, 2)
        n = neighbor[0]
        dataset.loc[dataset["id"] == o.row["id"], "neighbor_dist"] = n[1]
        dataset.loc[dataset["id"] == o.row["id"], "neighbor_total_price"] = n[0].row["per_square_meter_price"] * \
                                                                            n[0].row["total_square"]
        dataset.loc[dataset["id"] == o.row["id"], "neighbor_square_price"] = n[0].row["per_square_meter_price"]
        dataset.loc[dataset["id"] == o.row["id"], "neighbor10_dist"] = neighbor[10][1]


In [6]:

dataset=preprocess_floor.preprocess(dataset)



In [7]:
dataset_copy = dataset.copy()

In [8]:
dataset.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,date,realty_type,price_type,is_train,neighbor_dist,neighbor_total_price,neighbor_square_price,neighbor10_dist,has_basement,floor_count
0,Красноярск,-999,COL_62,56.063615,92.958428,0,7,14,26,0,...,2020-01-05,110,1,1,0.334024,995000.0,41458.333333,0.451369,-999,-999
1,Саратов,-999,COL_71,51.534581,46.020549,13,198,345,462,0,...,2020-01-05,10,1,1,0.086136,2985000.0,33166.666667,0.190652,-999,-999
2,Красноярск,-999,COL_140,56.026884,92.818323,3,15,23,33,0,...,2020-01-05,10,1,1,0.027117,18308000.0,61026.666667,0.291762,-999,-999
3,Иркутск,-999,COL_202,52.275528,104.251444,0,10,26,40,0,...,2020-01-05,10,1,1,0.220089,5870000.0,58700.000000,0.435699,-999,-999
4,Белгород,-999,COL_207,50.576545,36.584197,4,48,73,92,0,...,2020-01-05,10,1,1,0.046677,4179000.0,59700.000000,0.147191,-999,-999


In [9]:
def encode_categorical_features(df, categorical_columns):
    for column in categorical_columns:
        dict_encoding = {key: val for val, key in enumerate(df[column].unique())}
        df[column] = df[column].map(dict_encoding)
    return df

In [10]:

# Квантильное преобразование данных
def get_quantile_transform(_df, columns_for_quantilization, random_state=41, n_quantiles=100,
                           output_distribution='normal'):
    df = _df.copy()
    for col in columns_for_quantilization:
        qt = QuantileTransformer(random_state=random_state, n_quantiles=n_quantiles,
                                 output_distribution=output_distribution)
        df[col] = qt.fit_transform(df[[col]])
    return df

In [11]:

# МинМакс преобразование данных
def get_minmax_transform(_df, columns_for_quantilization, min_value=-1, max_value=1):
    df = _df.copy()
    for col in columns_for_quantilization:
        scaler = MinMaxScaler(feature_range=(min_value, max_value))
        df[col] = scaler.fit_transform(df[[col]])
    return df

In [12]:
# Hotencoding для категориальных фичей
data = encode_categorical_features(dataset, CATEGORICAL_FEATURES_COLUMNS)
# Нормализация численных данных
data = get_quantile_transform(data, NUM_FEATURES_COLUMNS)
data = get_minmax_transform(data, NUM_FEATURES_COLUMNS)
# Заполняем NaN значения
data = data.fillna(data.mean())
train = data[data.is_train == 1].reset_index(drop=True)
test = data[data.is_train == 0].reset_index(drop=True)
train = train.drop(columns=['is_train'])
test = test.drop(columns=['is_train'])

In [13]:
data.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,date,realty_type,price_type,is_train,neighbor_dist,neighbor_total_price,neighbor_square_price,neighbor10_dist,has_basement,floor_count
0,0,0,COL_62,0.060226,0.223088,-1.000000,-0.234768,-0.256798,-0.245381,-1.0,...,2020-01-05,0,1,1,0.295130,-0.335815,-0.100331,0.117364,-1.0,-1.0
1,1,0,COL_71,-0.284322,-0.042332,0.298058,0.285412,0.284259,0.272468,-1.0,...,2020-01-05,1,1,1,-0.125504,-0.153689,-0.153689,-0.153382,-1.0,-1.0
2,0,0,COL_140,0.033880,0.146323,0.067077,-0.131259,-0.182216,-0.206562,-1.0,...,2020-01-05,1,1,1,-0.370964,0.156685,0.014726,-0.012172,-1.0,-1.0
3,2,0,COL_202,-0.206973,0.271530,-1.000000,-0.194000,-0.167492,-0.178437,-1.0,...,2020-01-05,1,1,1,0.175130,-0.024577,0.001980,0.108546,-1.0,-1.0
4,3,0,COL_207,-0.335843,-0.118494,0.110487,0.077531,0.017066,-0.039228,-1.0,...,2020-01-05,1,1,1,-0.260458,-0.093665,0.007306,-0.236691,-1.0,-1.0


In [14]:
def get_standart_split(data, n_splits=5, seed=41):
    kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
    split_list = []
    for train_index, test_index in kf.split(data):
        split_list += [(train_index, test_index)]
    return split_list

In [15]:
def get_dataset(arr_features, arr_target, arr_region, arr_city, arr_realty, batch_size):
    return tf.data.Dataset.from_tensor_slices(
        (
            {
                "model_features_input": arr_features,
                "model_region_input": arr_region,
                "model_city_input": arr_city,
                "model_realty_input": arr_realty,
            },
            {
                "model_output": arr_target,
            },
        )
    ).batch(batch_size)

In [16]:
def get_columns_order(columns):
    columns_order = sorted([x for x in columns if not x in (CATEGORICAL_FEATURES_COLUMNS + TARGET_COLUMNS)])
    return columns_order + CATEGORICAL_FEATURES_COLUMNS + TARGET_COLUMNS

In [17]:

# Коллбэк, для отслеживания целевой метрики
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, val_dataset, val_targets):
        super(CustomCallback, self).__init__()
        self.val_targets = val_targets
        self.val_dataset = val_dataset

    def on_epoch_end(self, epoch, logs=None):
        predicts = self.model.predict(self.val_dataset)[:, 0]
        targets = self.val_targets[:, 0]
        print(f"Текущий реальный скор(валидационная часть): {np.round(deviation_metric(targets, predicts), 4)}")

In [18]:

def Dropout(x):
    return keras.layers.Dropout(x)


def Flatten():
    return keras.layers.Flatten()


def Concatenate():
    return keras.layers.Concatenate()


# Функция обучения модели
def fit(model, epochs, train_dataset, val_dataset, val_targets, verbose=True):
    if IS_GPU:
        print(f"Начинаю обучение модели (GPU) количество эпох = {epochs}")
        with tf.device('/device:GPU:0'):
            # Коллбэк для остановки, если модель перестала обучаться
            early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=2.5e-6,
                                                                       patience=100, restore_best_weights=True,
                                                                       mode='min')
            # Коллбэк для уменьшения скорости обучения
            lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-9,
                                                               mode='min')
            # Кастомный коллбэк для отображения скора по целевой метрике
            metric_callback = CustomCallback(val_dataset, val_targets)
            history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, verbose=verbose,
                                shuffle=True, callbacks=[early_stopping_callback, lr_callback, metric_callback],
                                workers=-1)
            return history
    else:
        print(f"Начинаю обучение модели (СPU) количество эпох = {epochs}")
        # Коллбэк для остановки, если модель перестала обучаться
        early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=2.5e-6, patience=100,
                                                                   restore_best_weights=True, mode='min')
        # Коллбэк для уменьшения скорости обучения
        lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-9,
                                                           mode='min')
        # Кастомный коллбэк для отображения скора по целевой метрике
        metric_callback = CustomCallback(val_dataset, val_targets)
        history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, verbose=verbose, shuffle=True,
                            callbacks=[early_stopping_callback, lr_callback, metric_callback], workers=-1)
        return history

In [19]:

# Реализация кастомной функции потерь для обучения
def tf_custom_loss(y_true, y_pred):
    threshold = 0.6
    error = tf.abs(y_true - y_pred) / y_true
    is_small_error = error <= threshold
    small_error_loss = tf.square(error / 0.15 - 1)
    big_error_loss = 9.0 * tf.ones_like(small_error_loss) + tf.abs(error)
    # big_error_loss = (3.0 * tf.ones_like(small_error_loss) + tf.abs(error)) ** 2
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [20]:

# Компиляция текущей модели
def compile_model(train_dataset, val_dataset, num_features, max_realty, max_region, max_city, lr=5e-4):
    reset_tensorflow_session()
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model_input_layer = tf.keras.Input(shape=(num_features), name="model_features_input")
    model_input_realty = tf.keras.Input(shape=(1), name="model_realty_input")
    model_input_region = tf.keras.Input(shape=(1), name="model_region_input")
    model_input_city = tf.keras.Input(shape=(1), name="model_city_input")

    model_embedding_layer_realty = keras.layers.Embedding(max_realty + 1, 4, input_length=1, dtype=tf.float64)(
        model_input_realty)
    model_embedding_layer_region = keras.layers.Embedding(max_region + 1, 32, input_length=1, dtype=tf.float64)(
        model_input_region)
    model_embedding_layer_city = keras.layers.Embedding(max_city + 1, 32, input_length=1, dtype=tf.float64)(
        model_input_city)

    concatenated_input_layer = Concatenate()(
        [Flatten()(model_embedding_layer_realty), Flatten()(model_embedding_layer_region),
         Flatten()(model_embedding_layer_city), Flatten()(model_input_layer)])

    layer_0 = keras.layers.Dense(128, activation="relu")(concatenated_input_layer)
    layer_1 = keras.layers.Dense(64, activation="relu")(layer_0)
    layer_2 = keras.layers.Dense(32, activation="relu")(layer_1)
    model_output_layer = keras.layers.Dense(1, activation="relu", name="model_output")(layer_2)

    cur_model = keras.Model(
        inputs=[
            model_input_layer,
            model_input_realty,
            model_input_region,
            model_input_city,
        ],
        outputs=[
            model_output_layer,
        ])

    print(f"Модель: input_shape = {cur_model.input_shape} output_shape = {cur_model.output_shape}")
#     cur_model.compile(loss=tf_custom_loss, optimizer=optimizer)  # , run_eagerly=True)
    cur_model.compile(loss=tf_custom_loss, optimizer=optimizer)  # , run_eagerly=True)

    #
    return cur_model

In [21]:
features_columns_order = get_columns_order(train.columns.values.tolist())
split_list = get_standart_split(train, n_splits=20)

start_train_model_time = time.time()
# Размер батча для Dataset
BATCH_SIZE = int(2 ** 5)
# Количество эпох обучения
EPOCHS = 500
# Количество численных входных переменных модели
NUM_FEATURES = len(NUM_FEATURES_COLUMNS)
# Макс. значения категориалных фичей
MAX_REALTY = max(train['realty_type'].max(), test['realty_type'].max())
MAX_REGION = max(train['region'].max(), test['region'].max())
MAX_CITY = max(train['city'].max(), test['city'].max())
# Коэффициент домножения таргета, с целью быстрейшего сходимости модельки и лучшего обучения
MUL_TARGET = 5e-5

scores = []
nn_predicts = np.zeros(len(train))
models_nn = []

for fold_num, (train_indexes, valid_indexes) in enumerate(split_list):
    start_time = time.time()
    print(f"Фолд: {fold_num}")

    train_sub_df = train[features_columns_order].loc[train_indexes].reset_index(drop=True)
    valid_sub_df = train[features_columns_order].loc[valid_indexes].reset_index(drop=True)

    print(f"Размер трейна = {train_sub_df.shape} Размер валидации = {valid_sub_df.shape}")

    # Строим датасеты
    train_ds = get_dataset(
        train_sub_df[NUM_FEATURES_COLUMNS].values,
        train_sub_df[TARGET_COLUMNS].values * MUL_TARGET,
        train_sub_df[['region']].values,
        train_sub_df[['city']].values,
        train_sub_df[['realty_type']].values,
        BATCH_SIZE)
    valid_ds = get_dataset(
        valid_sub_df[NUM_FEATURES_COLUMNS].values,
        valid_sub_df[TARGET_COLUMNS].values * MUL_TARGET,
        valid_sub_df[['region']].values,
        valid_sub_df[['city']].values,
        valid_sub_df[['realty_type']].values,
        len(valid_sub_df))

    # Компилируем модель
    model = compile_model(train_ds, valid_ds, NUM_FEATURES, MAX_REALTY, MAX_REGION, MAX_CITY)
    # Обучаем модель
    fit(model, EPOCHS, train_ds, valid_ds, valid_sub_df[TARGET_COLUMNS].values * MUL_TARGET)

    predict_on_validation = model.predict(valid_ds)[:, 0] / MUL_TARGET
    nn_predicts[valid_indexes] = predict_on_validation
    targets_for_validation = valid_sub_df[TARGET_COLUMNS].values[:, 0]
    current_score = deviation_metric(targets_for_validation, predict_on_validation)
    scores += [current_score]
    models_nn += [model]
    print(
        f"Скор для фолда({fold_num}) : {np.round(current_score, 4)} средний скор на префиксе = {np.round(np.mean(scores), 4)} это заняло = {int(time.time() - start_time)} сек.")
print(f"Процесс обучения модели занял = {int(time.time() - start_train_model_time)} секунд")

Фолд: 0
Размер трейна = (4268, 83) Размер валидации = (225, 83)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing

Epoch 26/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7501 - val_loss: 1.4693 - lr: 5.0000e-04
Epoch 27/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7963 - val_loss: 1.4550 - lr: 5.0000e-04
Epoch 28/500
134/134 [==============================] - 0s 2ms/step - loss: 0.8386 - val_loss: 1.4057 - lr: 5.0000e-04
Epoch 29/500
134/134 [==============================] - 0s 2ms/step - loss: 0.8048 - val_loss: 1.4615 - lr: 5.0000e-04
Epoch 30/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7559 - val_loss: 1.3601 - lr: 5.0000e-04
Epoch 31/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7816 - val_loss: 1.3522 - lr: 5.0000e-04
Epoch 32/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7709 - val_loss: 1.4292 - lr: 5.0000e-04
Epoch 33/500
134/134 [==============================] - 0s 2ms/step - loss: 0.8287 - val_loss: 1.4933 - lr: 5.0000e-04
Epoch 34/500
134/134 [==========================

Epoch 61/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4151 - val_loss: 1.2907 - lr: 2.5000e-04
Epoch 62/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4108 - val_loss: 1.3006 - lr: 2.5000e-04
Epoch 63/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4075 - val_loss: 1.3045 - lr: 2.5000e-04
Epoch 64/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4269 - val_loss: 1.3259 - lr: 1.2500e-04
Epoch 65/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4359 - val_loss: 1.3225 - lr: 1.2500e-04
Epoch 66/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4485 - val_loss: 1.3360 - lr: 1.2500e-04
Epoch 67/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4571 - val_loss: 1.3719 - lr: 1.2500e-04
Epoch 68/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4827 - val_loss: 1.4426 - lr: 1.2500e-04
Epoch 69/500
134/134 [==========================

Epoch 96/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3473 - val_loss: 1.2820 - lr: 1.5625e-05
Epoch 97/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3465 - val_loss: 1.2819 - lr: 1.5625e-05
Epoch 98/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3458 - val_loss: 1.2821 - lr: 1.5625e-05
Epoch 99/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3452 - val_loss: 1.2820 - lr: 1.5625e-05
Epoch 100/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3446 - val_loss: 1.2824 - lr: 1.5625e-05
Epoch 101/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3440 - val_loss: 1.2824 - lr: 1.5625e-05
Epoch 102/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3434 - val_loss: 1.2827 - lr: 1.5625e-05
Epoch 103/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3428 - val_loss: 1.2826 - lr: 1.5625e-05
Epoch 104/500
134/134 [=====================

Epoch 131/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3427 - val_loss: 1.3000 - lr: 1.9531e-06
Epoch 132/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3425 - val_loss: 1.3000 - lr: 1.9531e-06
Epoch 133/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3424 - val_loss: 1.3001 - lr: 1.9531e-06
Epoch 134/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3421 - val_loss: 1.3023 - lr: 9.7656e-07
Epoch 135/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3416 - val_loss: 1.3033 - lr: 9.7656e-07
Epoch 136/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3415 - val_loss: 1.3037 - lr: 9.7656e-07
Epoch 137/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3413 - val_loss: 1.3038 - lr: 9.7656e-07
Epoch 138/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3413 - val_loss: 1.3039 - lr: 9.7656e-07
Epoch 139/500
134/134 [=================

...

Epoch 106/500
134/134 [==============================] - 0s 2ms/step - loss: 0.5316 - val_loss: 1.4943 - lr: 9.7656e-07
Скор для фолда(18) : 1.2607 средний скор на префиксе = 1.3685 это заняло = 26 сек.
Фолд: 19
Размер трейна = (4269, 83) Размер валидации = (224, 83)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by d

Epoch 25/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7557 - val_loss: 1.5054 - lr: 5.0000e-04
Epoch 26/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7901 - val_loss: 1.4626 - lr: 5.0000e-04
Epoch 27/500
134/134 [==============================] - 0s 2ms/step - loss: 0.8217 - val_loss: 1.3804 - lr: 5.0000e-04
Epoch 28/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7641 - val_loss: 1.3879 - lr: 5.0000e-04
Epoch 29/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7627 - val_loss: 1.4329 - lr: 5.0000e-04
Epoch 30/500
134/134 [==============================] - 0s 2ms/step - loss: 0.8083 - val_loss: 1.4480 - lr: 5.0000e-04
Epoch 31/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7621 - val_loss: 1.6834 - lr: 5.0000e-04
Epoch 32/500
134/134 [==============================] - 0s 2ms/step - loss: 0.7734 - val_loss: 1.3643 - lr: 5.0000e-04
Epoch 33/500
134/134 [==========================

Epoch 60/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4156 - val_loss: 1.3351 - lr: 1.2500e-04
Epoch 61/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4081 - val_loss: 1.3319 - lr: 1.2500e-04
Epoch 62/500
134/134 [==============================] - 0s 2ms/step - loss: 0.4006 - val_loss: 1.3345 - lr: 1.2500e-04
Epoch 63/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3953 - val_loss: 1.3314 - lr: 1.2500e-04
Epoch 64/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3905 - val_loss: 1.3353 - lr: 1.2500e-04
Epoch 65/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3861 - val_loss: 1.3364 - lr: 1.2500e-04
Epoch 66/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3961 - val_loss: 1.3256 - lr: 6.2500e-05
Epoch 67/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3937 - val_loss: 1.3257 - lr: 6.2500e-05
Epoch 68/500
134/134 [==========================

Epoch 95/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3622 - val_loss: 1.3407 - lr: 1.5625e-05
Epoch 96/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3623 - val_loss: 1.3368 - lr: 7.8125e-06
Epoch 97/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3616 - val_loss: 1.3364 - lr: 7.8125e-06
Epoch 98/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3608 - val_loss: 1.3366 - lr: 7.8125e-06
Epoch 99/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3602 - val_loss: 1.3367 - lr: 7.8125e-06
Epoch 100/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3595 - val_loss: 1.3369 - lr: 7.8125e-06
Epoch 101/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3589 - val_loss: 1.3373 - lr: 7.8125e-06
Epoch 102/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3583 - val_loss: 1.3375 - lr: 7.8125e-06
Epoch 103/500
134/134 [======================

Epoch 130/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3497 - val_loss: 1.3338 - lr: 9.7656e-07
Epoch 131/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3496 - val_loss: 1.3338 - lr: 9.7656e-07
Epoch 132/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3496 - val_loss: 1.3337 - lr: 9.7656e-07
Epoch 133/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3495 - val_loss: 1.3337 - lr: 9.7656e-07
Epoch 134/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3495 - val_loss: 1.3336 - lr: 9.7656e-07
Epoch 135/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3494 - val_loss: 1.3335 - lr: 9.7656e-07
Epoch 136/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3489 - val_loss: 1.3336 - lr: 4.8828e-07
Epoch 137/500
134/134 [==============================] - 0s 2ms/step - loss: 0.3489 - val_loss: 1.3337 - lr: 4.8828e-07
Epoch 138/500
134/134 [=================

In [22]:
# Предикт нейронной сетью на test
def get_nn_predict(models, test):
    result = np.zeros(len(test))
    test_ds = get_dataset(
        test[NUM_FEATURES_COLUMNS].values,
        np.zeros(len(test)),
        test[['region']].values,
        test[['city']].values,
        test[['realty_type']].values,
        len(test))
    for model in models:
        predict = model.predict(test_ds)[:, 0]
        result += (predict / MUL_TARGET) / len(models)
    return result


test_nn_predict = get_nn_predict(models_nn, test)

test_submission = pd.read_csv('dataset/test_submission.csv')

test_submission['per_square_meter_price'] = test_nn_predict
test_submission.to_csv('nn2.csv', index=False)

In [36]:

# LightGBM кастомная метрика
def feval_deviation(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'deviation_error', deviation_metric(np.exp(y_true), np.exp(y_pred)/1.1), False


# Функция для обучения модели LightGBM
def train_lgb(train, valid, num_features, categorical_features, target_train, target_valid, EPOCHS, params):
    # feature_importances = np.zeros(len(features))
    train_dataset = lgb.Dataset(train[num_features + categorical_features], np.log(target_train), 
                                categorical_feature=categorical_features)
    valid_dataset = lgb.Dataset(valid[num_features + categorical_features], np.log(target_valid), 
#                                 weight=(1.0 / target_valid),
                                categorical_feature=categorical_features)
    model = lgb.train(
        params=params,
        num_boost_round=EPOCHS,
        train_set=train_dataset,
        valid_sets=[train_dataset, valid_dataset],
        verbose_eval=100,
        early_stopping_rounds=int(5 / params['learning_rate']),
        feval=feval_deviation)

    y_valid = model.predict(valid[num_features + categorical_features])
    # feature_importances = model.feature_importance(importance_type='gain') / 5.0
    # lgb.plot_importance(model,max_num_features = 41)

    return model, np.exp(y_valid)


start_train_model_time = time.time()

boosting_seed = 41
boosting_params = {
    'bagging_fraction': 0.9,
    'bagging_freq': 1,
    'boost': 'gbdt',
    'feature_fraction': 0.9,
    'max_depth': 4,
    'learning_rate': 0.03,
    'metric': 'custom',
    'objective': 'regression_l1',
    'verbose': -1,
    'n_jobs': -1,
    'seed': boosting_seed,
    'feature_fraction_seed': boosting_seed,
    'bagging_seed': boosting_seed,
    'drop_seed': boosting_seed,
    'data_random_seed': boosting_seed,
}

# Количество эпох обучения
EPOCHS = 10000
scores = []
lgb_predicts = np.zeros(len(train))

lgb_models = []
for fold_num, (train_indexes, valid_indexes) in enumerate(split_list):
    start_time = time.time()
    print(f"Фолд: {fold_num}")

    train_sub_df = train[features_columns_order].loc[train_indexes].reset_index(drop=True)
    valid_sub_df = train[features_columns_order].loc[valid_indexes].reset_index(drop=True)

    print(f"Размер трейна = {train_sub_df.shape} Размер валидации = {valid_sub_df.shape}")
    # Обучаем LightGBM и делаем предикт на валидационной выборке
    model, predict_validation = train_lgb(
        train_sub_df,
        valid_sub_df,
        NUM_FEATURES_COLUMNS,
        CATEGORICAL_FEATURES_COLUMNS,
        train_sub_df[TARGET_COLUMNS[0]].values,
        valid_sub_df[TARGET_COLUMNS[0]].values,
        EPOCHS,
        boosting_params)

    lgb_models += [model]
    predict_on_validation = model.predict(valid_sub_df[NUM_FEATURES_COLUMNS + CATEGORICAL_FEATURES_COLUMNS])
    lgb_predicts[valid_indexes] = np.exp(predict_on_validation)
    targets_for_validation = valid_sub_df[TARGET_COLUMNS].values[:, 0]
    current_score = deviation_metric(targets_for_validation, predict_on_validation)
    scores += [current_score]
    print(
        f"Скор для фолда({fold_num}) : {np.round(current_score, 4)} средний скор на префиксе = {np.round(np.mean(scores), 4)} это заняло = {int(time.time() - start_time)} сек.")
print(f"Процесс обучения модели занял = {int(time.time() - start_train_model_time)} секунд")

Фолд: 0
Размер трейна = (4268, 83) Размер валидации = (225, 83)
Training until validation scores don't improve for 166 rounds
[100]	training's deviation_error: 1.24152	valid_1's deviation_error: 1.51488
[200]	training's deviation_error: 1.00517	valid_1's deviation_error: 1.37652
[300]	training's deviation_error: 0.90895	valid_1's deviation_error: 1.33996
[400]	training's deviation_error: 0.84622	valid_1's deviation_error: 1.32104
[500]	training's deviation_error: 0.795471	valid_1's deviation_error: 1.30655
[600]	training's deviation_error: 0.763442	valid_1's deviation_error: 1.29798
[700]	training's deviation_error: 0.732594	valid_1's deviation_error: 1.28593
[800]	training's deviation_error: 0.709994	valid_1's deviation_error: 1.28508
[900]	training's deviation_error: 0.691849	valid_1's deviation_error: 1.27828
[1000]	training's deviation_error: 0.675379	valid_1's deviation_error: 1.27753
[1100]	training's deviation_error: 0.660379	valid_1's deviation_error: 1.26875
[1200]	training's 

[2100]	training's deviation_error: 0.590709	valid_1's deviation_error: 1.11698
[2200]	training's deviation_error: 0.583778	valid_1's deviation_error: 1.11528
[2300]	training's deviation_error: 0.578963	valid_1's deviation_error: 1.11102
[2400]	training's deviation_error: 0.57241	valid_1's deviation_error: 1.10991
[2500]	training's deviation_error: 0.567298	valid_1's deviation_error: 1.10578
[2600]	training's deviation_error: 0.562303	valid_1's deviation_error: 1.10409
[2700]	training's deviation_error: 0.557586	valid_1's deviation_error: 1.10377
Early stopping, best iteration is:
[2629]	training's deviation_error: 0.561344	valid_1's deviation_error: 1.10268
Скор для фолда(3) : 9.0 средний скор на префиксе = 9.0 это заняло = 67 сек.
Фолд: 4
Размер трейна = (4268, 83) Размер валидации = (225, 83)
Training until validation scores don't improve for 166 rounds
[100]	training's deviation_error: 1.25062	valid_1's deviation_error: 1.49469
[200]	training's deviation_error: 1.01512	valid_1's dev

[2800]	training's deviation_error: 0.550136	valid_1's deviation_error: 1.11889
[2900]	training's deviation_error: 0.545386	valid_1's deviation_error: 1.11691
[3000]	training's deviation_error: 0.542543	valid_1's deviation_error: 1.11433
[3100]	training's deviation_error: 0.539008	valid_1's deviation_error: 1.11014
[3200]	training's deviation_error: 0.535287	valid_1's deviation_error: 1.10979
[3300]	training's deviation_error: 0.531793	valid_1's deviation_error: 1.10914
[3400]	training's deviation_error: 0.528306	valid_1's deviation_error: 1.10789
[3500]	training's deviation_error: 0.525941	valid_1's deviation_error: 1.10662
[3600]	training's deviation_error: 0.522652	valid_1's deviation_error: 1.10385
[3700]	training's deviation_error: 0.519867	valid_1's deviation_error: 1.10423
[3800]	training's deviation_error: 0.516957	valid_1's deviation_error: 1.10197
[3900]	training's deviation_error: 0.514725	valid_1's deviation_error: 1.10077
[4000]	training's deviation_error: 0.512087	valid_1'

[100]	training's deviation_error: 1.23892	valid_1's deviation_error: 1.79407
[200]	training's deviation_error: 0.985783	valid_1's deviation_error: 1.52817
[300]	training's deviation_error: 0.899394	valid_1's deviation_error: 1.44138
[400]	training's deviation_error: 0.845258	valid_1's deviation_error: 1.4001
[500]	training's deviation_error: 0.803106	valid_1's deviation_error: 1.36066
[600]	training's deviation_error: 0.774159	valid_1's deviation_error: 1.34057
[700]	training's deviation_error: 0.749662	valid_1's deviation_error: 1.32329
[800]	training's deviation_error: 0.725621	valid_1's deviation_error: 1.31042
[900]	training's deviation_error: 0.706279	valid_1's deviation_error: 1.30182
[1000]	training's deviation_error: 0.688198	valid_1's deviation_error: 1.29427
[1100]	training's deviation_error: 0.670767	valid_1's deviation_error: 1.28849
[1200]	training's deviation_error: 0.658176	valid_1's deviation_error: 1.28701
[1300]	training's deviation_error: 0.645009	valid_1's deviation

[600]	training's deviation_error: 0.780037	valid_1's deviation_error: 1.04759
[700]	training's deviation_error: 0.753476	valid_1's deviation_error: 1.03437
[800]	training's deviation_error: 0.728477	valid_1's deviation_error: 1.021
[900]	training's deviation_error: 0.709142	valid_1's deviation_error: 1.01337
[1000]	training's deviation_error: 0.69251	valid_1's deviation_error: 1.00744
[1100]	training's deviation_error: 0.6766	valid_1's deviation_error: 1.00125
[1200]	training's deviation_error: 0.662214	valid_1's deviation_error: 0.996647
[1300]	training's deviation_error: 0.649438	valid_1's deviation_error: 0.993539
[1400]	training's deviation_error: 0.638754	valid_1's deviation_error: 0.990488
[1500]	training's deviation_error: 0.630126	valid_1's deviation_error: 0.987368
[1600]	training's deviation_error: 0.621695	valid_1's deviation_error: 0.989218
[1700]	training's deviation_error: 0.612588	valid_1's deviation_error: 0.986113
[1800]	training's deviation_error: 0.605466	valid_1's d

In [37]:

# Предикт lgb на test
def get_lgb_predict(models, test):
    result = np.zeros(len(test))
    for model in models:
        predict = model.predict(test[NUM_FEATURES_COLUMNS + CATEGORICAL_FEATURES_COLUMNS])
        result += np.exp(predict) / len(models)
    return result


test_lgb_predict = get_lgb_predict(lgb_models, test)

test_lgb_predict.min(), test_lgb_predict.max(), test_lgb_predict.mean()

(17235.261783495247, 459168.7004751519, 60440.58801072709)

In [58]:

# Кастомная метрика для xgboost
def xbg_error(preds, dtrain):
    labels = dtrain.get_label()
    err = deviation_metric(np.exp(labels), np.exp(preds)/1.05)
    return 'deviation_error', err


def train_xgb(train, valid, num_features, categorical_features, target_train, target_valid, EPOCHS, params):
    dtest = xgb.DMatrix(test[num_features + categorical_features])
    y_valid = np.zeros(len(valid))

    dtrain = xgb.DMatrix(train[num_features + categorical_features], np.log(target_train), 
                        )
    dvalid = xgb.DMatrix(valid[num_features + categorical_features], np.log(target_valid), 
                        )
    model = xgb.train(
        params,
        dtrain,
        EPOCHS,
        [(dvalid, "valid")],
        verbose_eval=250,
        early_stopping_rounds=500,
        feval=xbg_error,
    )
    y_valid = model.predict(dvalid)

    return model, y_valid


start_train_model_time = time.time()

xgboost_seed = 41
xgboost_params = {
    "subsample": 0.70,
    "colsample_bytree": 0.50,
    "max_depth": 7,
    "learning_rate": 0.012,
    "objective": "reg:squarederror",
    'disable_default_eval_metric': 1,
    "nthread": -1,
    "max_bin": 128,
    'min_child_weight': 0.0,
    'reg_lambda': 0.0,
    'reg_alpha': 0.0,
    'seed': xgboost_seed,
}

# Количество эпох обучения
EPOCHS = 10000
scores = []
xgb_predicts = np.zeros(len(train))

xgb_models = []
for fold_num, (train_indexes, valid_indexes) in enumerate(split_list):
    start_time = time.time()
    print(f"Фолд: {fold_num}")

    train_sub_df = train[features_columns_order].loc[train_indexes].reset_index(drop=True)
    valid_sub_df = train[features_columns_order].loc[valid_indexes].reset_index(drop=True)

    print(f"Размер трейна = {train_sub_df.shape} Размер валидации = {valid_sub_df.shape}")
    # Обучаем Xgboost и делаем предикт на валидационной выборке
    model, predict_validation = train_xgb(
        train_sub_df,
        valid_sub_df,
        NUM_FEATURES_COLUMNS,
        CATEGORICAL_FEATURES_COLUMNS,
        train_sub_df[TARGET_COLUMNS[0]].values,
        valid_sub_df[TARGET_COLUMNS[0]].values,
        EPOCHS,
        xgboost_params)

    xgb_models += [model]
    predict_on_validation = model.predict(
        xgb.DMatrix(valid_sub_df[NUM_FEATURES_COLUMNS + CATEGORICAL_FEATURES_COLUMNS]))
    xgb_predicts[valid_indexes] = np.exp(predict_on_validation)
    targets_for_validation = valid_sub_df[TARGET_COLUMNS].values[:, 0]
    current_score = deviation_metric(targets_for_validation, predict_on_validation)
    scores += [current_score]
    print(
        f"Скор для фолда({fold_num}) : {np.round(current_score, 4)} средний скор на префиксе = {np.round(np.mean(scores), 4)} это заняло = {int(time.time() - start_time)} сек.")
print(f"Процесс обучения модели занял = {int(time.time() - start_train_model_time)} секунд")






Фолд: 0
Размер трейна = (4268, 83) Размер валидации = (225, 83)
[0]	valid-deviation_error:9.00000
[250]	valid-deviation_error:3.79933
[500]	valid-deviation_error:1.20866
[750]	valid-deviation_error:1.18503
[1000]	valid-deviation_error:1.14266
[1250]	valid-deviation_error:1.12162
[1500]	valid-deviation_error:1.11663
[1750]	valid-deviation_error:1.10325
[2000]	valid-deviation_error:1.09861
[2250]	valid-deviation_error:1.09426
[2500]	valid-deviation_error:1.08948
[2750]	valid-deviation_error:1.08501
[3000]	valid-deviation_error:1.08590
[3250]	valid-deviation_error:1.08481
[3359]	valid-deviation_error:1.08481
Скор для фолда(0) : 9.0 средний скор на префиксе = 9.0 это заняло = 25 сек.
Фолд: 1
Размер трейна = (4268, 83) Размер валидации = (225, 83)
[0]	valid-deviation_error:9.00000
[250]	valid-deviation_error:3.92815
[500]	valid-deviation_error:1.08376
[750]	valid-deviation_error:1.06931
[1000]	valid-deviation_error:1.04807
[1250]	valid-deviation_error:1.03123
[1500]	valid-deviation_error:1.

[250]	valid-deviation_error:3.83868
[500]	valid-deviation_error:0.97363
[750]	valid-deviation_error:0.97674
[1000]	valid-deviation_error:0.95682
[1250]	valid-deviation_error:0.94234
[1500]	valid-deviation_error:0.94216
[1750]	valid-deviation_error:0.94449
[1803]	valid-deviation_error:0.94351
Скор для фолда(18) : 9.0 средний скор на префиксе = 9.0 это заняло = 14 сек.
Фолд: 19
Размер трейна = (4269, 83) Размер валидации = (224, 83)
[0]	valid-deviation_error:9.00000
[250]	valid-deviation_error:4.12486
[500]	valid-deviation_error:0.97530
[750]	valid-deviation_error:0.88909
[1000]	valid-deviation_error:0.86000
[1250]	valid-deviation_error:0.84198
[1500]	valid-deviation_error:0.83646
[1750]	valid-deviation_error:0.82904
[2000]	valid-deviation_error:0.83065
[2250]	valid-deviation_error:0.83067
[2500]	valid-deviation_error:0.83510
[2651]	valid-deviation_error:0.83728
Скор для фолда(19) : 9.0 средний скор на префиксе = 9.0 это заняло = 20 сек.
Процесс обучения модели занял = 291 секунд


In [59]:
# Предикт xgb на test
def get_xgb_predict(models, test):
    result = np.zeros(len(test))
    for model in models:
        predict = model.predict(xgb.DMatrix(test[NUM_FEATURES_COLUMNS + CATEGORICAL_FEATURES_COLUMNS]))
        result += predict / len(models)
    return result

test_xgb_predict = get_xgb_predict(xgb_models, test)

test_xgb_predict=np.exp(test_xgb_predict)

test_xgb_predict.min(), test_xgb_predict.max(), test_xgb_predict.mean()

train_targets = train[TARGET_COLUMNS[0]].values

In [27]:
class CatBoostEvalMetricPearson(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        preds = np.array(approxes[0])
        target = np.array(target)
        err = deviation_metric(np.exp(target), np.exp(preds))
        return err, 0


def train_cat(train, valid, num_features, categorical_features, target_train, target_valid, EPOCHS):





    test_data = Pool(data=test[num_features + categorical_features],
                  cat_features=categorical_features)


    train_data = Pool(data=train[num_features + categorical_features],
                      cat_features=categorical_features,
                      label=np.log(target_train))

    val_data = Pool(data=valid[num_features + categorical_features],
                cat_features=categorical_features,
                  label=np.log(target_valid))
    

    cat_model = CatBoostRegressor(
        learning_rate=0.012,
        iterations=15000,
        metric_period=50,
        eval_metric=CatBoostEvalMetricPearson(),
    )
    cat_model.fit(train_data, eval_set=val_data, use_best_model=True, early_stopping_rounds=300)
  
    y_valid = cat_model.predict(test_data)

    return cat_model, y_valid


start_train_model_time = time.time()

scores = []
cat_predicts = np.zeros(len(train))

cat_models = []
for fold_num, (train_indexes, valid_indexes) in enumerate(split_list):
    start_time = time.time()
    print(f"Фолд: {fold_num}")

    train_sub_df = train[features_columns_order].loc[train_indexes].reset_index(drop=True)
    valid_sub_df = train[features_columns_order].loc[valid_indexes].reset_index(drop=True)

    print(f"Размер трейна = {train_sub_df.shape} Размер валидации = {valid_sub_df.shape}")
    model, predict_validation = train_cat(
        train_sub_df,
        valid_sub_df,
        NUM_FEATURES_COLUMNS,
        CATEGORICAL_FEATURES_COLUMNS,
        train_sub_df[TARGET_COLUMNS[0]].values,
        valid_sub_df[TARGET_COLUMNS[0]].values,
        EPOCHS
        )

    cat_models += [model]
    predict_on_validation = model.predict(valid_sub_df[NUM_FEATURES_COLUMNS + CATEGORICAL_FEATURES_COLUMNS])
    cat_predicts[valid_indexes] = np.exp(predict_on_validation)
    targets_for_validation = valid_sub_df[TARGET_COLUMNS].values[:, 0]
    current_score = deviation_metric(targets_for_validation, predict_on_validation)
    scores += [current_score]
    print(
        f"Скор для фолда({fold_num}) : {np.round(current_score, 4)} средний скор на префиксе = {np.round(np.mean(scores), 4)} это заняло = {int(time.time() - start_time)} сек.")
print(f"Процесс обучения модели занял = {int(time.time() - start_train_model_time)} секунд")



Фолд: 0
Размер трейна = (4268, 83) Размер валидации = (225, 83)
0:	learn: 3.6306521	test: 3.3506941	best: 3.3506941 (0)	total: 78.6ms	remaining: 19m 39s


50:	learn: 2.7437735	test: 2.5529870	best: 2.5529870 (50)	total: 408ms	remaining: 1m 59s
100:	learn: 2.2825363	test: 2.2303113	best: 2.2303113 (100)	total: 742ms	remaining: 1m 49s
150:	learn: 2.0246339	test: 2.0709647	best: 2.0709647 (150)	total: 1.07s	remaining: 1m 45s
200:	learn: 1.8561306	test: 2.0013843	best: 2.0013843 (200)	total: 1.4s	remaining: 1m 42s
250:	learn: 1.7419400	test: 1.9760976	best: 1.9760075 (246)	total: 1.74s	remaining: 1m 41s
300:	learn: 1.6564567	test: 1.9563171	best: 1.9563171 (300)	total: 2.07s	remaining: 1m 41s
350:	learn: 1.5903019	test: 1.9151384	best: 1.9151384 (350)	total: 2.39s	remaining: 1m 39s
400:	learn: 1.5368943	test: 1.8932605	best: 1.8932605 (400)	total: 2.73s	remaining: 1m 39s
450:	learn: 1.4940564	test: 1.8593936	best: 1.8593936 (450)	total: 3.05s	remaining: 1m 38s
500:	learn: 1.4559319	test: 1.8326903	best: 1.8326903 (500)	total: 3.38s	remaining: 1m 37s
550:	learn: 1.4227710	test: 1.8176235	best: 1.8147836 (547)	total: 3.71s	remaining: 1m 37s
60

4500:	learn: 0.4330121	test: 1.4333186	best: 1.4333186 (4500)	total: 29.9s	remaining: 1m 9s
4550:	learn: 0.4267195	test: 1.4331883	best: 1.4321611 (4525)	total: 30.2s	remaining: 1m 9s
4600:	learn: 0.4202779	test: 1.4334267	best: 1.4321611 (4525)	total: 30.5s	remaining: 1m 9s
4650:	learn: 0.4133502	test: 1.4315901	best: 1.4314675 (4648)	total: 30.9s	remaining: 1m 8s
4700:	learn: 0.4069490	test: 1.4318253	best: 1.4310959 (4652)	total: 31.2s	remaining: 1m 8s
4750:	learn: 0.4013808	test: 1.4288943	best: 1.4288943 (4750)	total: 31.5s	remaining: 1m 8s
4800:	learn: 0.3951921	test: 1.4265970	best: 1.4265970 (4800)	total: 31.9s	remaining: 1m 7s
4850:	learn: 0.3897914	test: 1.4262991	best: 1.4252903 (4809)	total: 32.2s	remaining: 1m 7s
4900:	learn: 0.3846513	test: 1.4245582	best: 1.4245582 (4900)	total: 32.5s	remaining: 1m 7s
4950:	learn: 0.3789320	test: 1.4219940	best: 1.4219940 (4950)	total: 32.9s	remaining: 1m 6s
5000:	learn: 0.3745140	test: 1.4210193	best: 1.4207749 (4997)	total: 33.2s	remai

9000:	learn: 0.1137623	test: 1.3287239	best: 1.3285460 (8993)	total: 60s	remaining: 40s
9050:	learn: 0.1121136	test: 1.3272415	best: 1.3270474 (9033)	total: 1m	remaining: 39.6s
9100:	learn: 0.1106201	test: 1.3267792	best: 1.3254563 (9085)	total: 1m	remaining: 39.3s
9150:	learn: 0.1092699	test: 1.3259285	best: 1.3254563 (9085)	total: 1m	remaining: 39s
9200:	learn: 0.1077485	test: 1.3250922	best: 1.3250922 (9200)	total: 1m 1s	remaining: 38.6s
9250:	learn: 0.1062121	test: 1.3236869	best: 1.3235176 (9247)	total: 1m 1s	remaining: 38.3s
9300:	learn: 0.1040683	test: 1.3225128	best: 1.3225128 (9300)	total: 1m 1s	remaining: 38s
9350:	learn: 0.1024694	test: 1.3220058	best: 1.3219963 (9348)	total: 1m 2s	remaining: 37.6s
9400:	learn: 0.1006324	test: 1.3196980	best: 1.3195212 (9399)	total: 1m 2s	remaining: 37.3s
9450:	learn: 0.0994663	test: 1.3177843	best: 1.3176847 (9449)	total: 1m 2s	remaining: 37s
9500:	learn: 0.0981823	test: 1.3184463	best: 1.3173065 (9457)	total: 1m 3s	remaining: 36.6s
9550:	l

50:	learn: 2.7328784	test: 2.5512925	best: 2.5512925 (50)	total: 370ms	remaining: 1m 48s
100:	learn: 2.2720424	test: 2.1555536	best: 2.1555536 (100)	total: 696ms	remaining: 1m 42s
150:	learn: 2.0197101	test: 1.9510163	best: 1.9510163 (150)	total: 1.03s	remaining: 1m 41s
200:	learn: 1.8594761	test: 1.8304153	best: 1.8304153 (200)	total: 1.37s	remaining: 1m 41s
250:	learn: 1.7475463	test: 1.7563831	best: 1.7563831 (250)	total: 1.72s	remaining: 1m 41s
300:	learn: 1.6611104	test: 1.7020631	best: 1.7020631 (300)	total: 2.06s	remaining: 1m 40s
350:	learn: 1.6018290	test: 1.6773239	best: 1.6773239 (350)	total: 2.4s	remaining: 1m 40s
400:	learn: 1.5477206	test: 1.6464018	best: 1.6464018 (400)	total: 2.72s	remaining: 1m 39s
450:	learn: 1.5043351	test: 1.6164146	best: 1.6164146 (450)	total: 3.05s	remaining: 1m 38s
500:	learn: 1.4690814	test: 1.5961961	best: 1.5961961 (500)	total: 3.38s	remaining: 1m 37s
550:	learn: 1.4375395	test: 1.5852692	best: 1.5847105 (539)	total: 3.71s	remaining: 1m 37s
60

...

0:	learn: 3.6226363	test: 3.2512653	best: 3.2512653 (0)	total: 27.7ms	remaining: 6m 54s
50:	learn: 2.7701160	test: 2.4071539	best: 2.4071539 (50)	total: 373ms	remaining: 1m 49s
100:	learn: 2.2987696	test: 1.9511365	best: 1.9511365 (100)	total: 707ms	remaining: 1m 44s
150:	learn: 2.0351416	test: 1.7848057	best: 1.7848057 (150)	total: 1.03s	remaining: 1m 41s
200:	learn: 1.8641011	test: 1.7153617	best: 1.7153617 (200)	total: 1.37s	remaining: 1m 40s
250:	learn: 1.7567307	test: 1.6494796	best: 1.6494796 (250)	total: 1.71s	remaining: 1m 40s
300:	learn: 1.6697135	test: 1.5942493	best: 1.5942493 (300)	total: 2.04s	remaining: 1m 39s
350:	learn: 1.6047347	test: 1.5487943	best: 1.5487943 (350)	total: 2.38s	remaining: 1m 39s
400:	learn: 1.5481952	test: 1.5249346	best: 1.5245599 (399)	total: 2.71s	remaining: 1m 38s
450:	learn: 1.5018355	test: 1.5036618	best: 1.5036618 (450)	total: 3.04s	remaining: 1m 37s
500:	learn: 1.4679591	test: 1.4882933	best: 1.4880972 (499)	total: 3.37s	remaining: 1m 37s
550:

4450:	learn: 0.4428876	test: 1.1136551	best: 1.1129978 (4377)	total: 29.8s	remaining: 1m 10s
4500:	learn: 0.4365678	test: 1.1134435	best: 1.1127145 (4488)	total: 30.1s	remaining: 1m 10s
4550:	learn: 0.4287317	test: 1.1123707	best: 1.1116823 (4539)	total: 30.5s	remaining: 1m 9s
4600:	learn: 0.4210880	test: 1.1114388	best: 1.1114388 (4600)	total: 30.8s	remaining: 1m 9s
4650:	learn: 0.4149891	test: 1.1102022	best: 1.1098077 (4641)	total: 31.2s	remaining: 1m 9s
4700:	learn: 0.4078046	test: 1.1076923	best: 1.1074898 (4693)	total: 31.5s	remaining: 1m 8s
4750:	learn: 0.4004519	test: 1.1045332	best: 1.1045332 (4750)	total: 31.8s	remaining: 1m 8s
4800:	learn: 0.3940221	test: 1.1028595	best: 1.1028595 (4800)	total: 32.2s	remaining: 1m 8s
4850:	learn: 0.3885097	test: 1.1017832	best: 1.1016565 (4823)	total: 32.5s	remaining: 1m 8s
4900:	learn: 0.3820128	test: 1.0992914	best: 1.0992317 (4894)	total: 32.8s	remaining: 1m 7s
4950:	learn: 0.3768590	test: 1.0988065	best: 1.0987361 (4946)	total: 33.2s	rem

50:	learn: 2.7352762	test: 2.5671417	best: 2.5671417 (50)	total: 374ms	remaining: 1m 49s
100:	learn: 2.2787922	test: 2.1113562	best: 2.1113562 (100)	total: 702ms	remaining: 1m 43s
150:	learn: 2.0229346	test: 1.8864193	best: 1.8864193 (150)	total: 1.04s	remaining: 1m 42s
200:	learn: 1.8627342	test: 1.7428976	best: 1.7428976 (200)	total: 1.37s	remaining: 1m 40s
250:	learn: 1.7548526	test: 1.6149316	best: 1.6149316 (250)	total: 1.7s	remaining: 1m 40s
300:	learn: 1.6694342	test: 1.5477663	best: 1.5477663 (300)	total: 2.05s	remaining: 1m 39s
350:	learn: 1.6049778	test: 1.4972588	best: 1.4972588 (350)	total: 2.38s	remaining: 1m 39s
400:	learn: 1.5517578	test: 1.4530110	best: 1.4530110 (400)	total: 2.71s	remaining: 1m 38s
450:	learn: 1.5089822	test: 1.4264345	best: 1.4264345 (450)	total: 3.04s	remaining: 1m 38s
500:	learn: 1.4696372	test: 1.4025201	best: 1.4025201 (500)	total: 3.37s	remaining: 1m 37s
550:	learn: 1.4377610	test: 1.3797218	best: 1.3794091 (549)	total: 3.7s	remaining: 1m 37s
600

4500:	learn: 0.4311398	test: 1.0178743	best: 1.0174173 (4495)	total: 30s	remaining: 1m 9s
4550:	learn: 0.4235883	test: 1.0170152	best: 1.0169787 (4549)	total: 30.3s	remaining: 1m 9s
4600:	learn: 0.4171337	test: 1.0157339	best: 1.0157339 (4600)	total: 30.6s	remaining: 1m 9s
4650:	learn: 0.4107953	test: 1.0137717	best: 1.0137717 (4650)	total: 31s	remaining: 1m 8s
4700:	learn: 0.4047021	test: 1.0109747	best: 1.0109747 (4700)	total: 31.3s	remaining: 1m 8s
4750:	learn: 0.3983622	test: 1.0084846	best: 1.0084846 (4750)	total: 31.6s	remaining: 1m 8s
4800:	learn: 0.3921534	test: 1.0078243	best: 1.0077275 (4797)	total: 31.9s	remaining: 1m 7s
4850:	learn: 0.3851365	test: 1.0058759	best: 1.0058095 (4849)	total: 32.3s	remaining: 1m 7s
4900:	learn: 0.3785457	test: 1.0035504	best: 1.0032622 (4889)	total: 32.6s	remaining: 1m 7s
4950:	learn: 0.3728824	test: 1.0037866	best: 1.0032622 (4889)	total: 32.9s	remaining: 1m 6s
5000:	learn: 0.3665324	test: 1.0031675	best: 1.0024041 (4979)	total: 33.3s	remaining

9000:	learn: 0.1022891	test: 0.9126410	best: 0.9126188 (8992)	total: 1m	remaining: 40.1s
9050:	learn: 0.1006868	test: 0.9131400	best: 0.9118745 (9022)	total: 1m	remaining: 39.8s
9100:	learn: 0.0990179	test: 0.9127250	best: 0.9118745 (9022)	total: 1m	remaining: 39.5s
9150:	learn: 0.0973472	test: 0.9122094	best: 0.9118745 (9022)	total: 1m 1s	remaining: 39.1s
9200:	learn: 0.0957610	test: 0.9108152	best: 0.9108152 (9200)	total: 1m 1s	remaining: 38.8s
9250:	learn: 0.0943379	test: 0.9114388	best: 0.9105874 (9214)	total: 1m 1s	remaining: 38.5s
9300:	learn: 0.0930154	test: 0.9107344	best: 0.9105725 (9295)	total: 1m 2s	remaining: 38.1s
9350:	learn: 0.0912766	test: 0.9103224	best: 0.9102588 (9347)	total: 1m 2s	remaining: 37.8s
9400:	learn: 0.0897928	test: 0.9098641	best: 0.9096248 (9394)	total: 1m 2s	remaining: 37.5s
9450:	learn: 0.0880381	test: 0.9092286	best: 0.9090008 (9443)	total: 1m 3s	remaining: 37.1s
9500:	learn: 0.0864351	test: 0.9080325	best: 0.9079264 (9492)	total: 1m 3s	remaining: 36.

In [28]:

# Предикт xgb на test
def get_cat_predict(models, test):
    result = np.zeros(len(test))
    for model in cat_models:
        predict = model.predict(test[NUM_FEATURES_COLUMNS + CATEGORICAL_FEATURES_COLUMNS])
        result += np.exp(predict) / len(models)
    return result


test_cat_predict = get_cat_predict(xgb_models, test)

test_cat_predict.min(), test_cat_predict.max(), test_cat_predict.mean()



(17326.68044258941, 1274188.5949374526, 61582.147919846844)

Optimization terminated successfully.
         Current function value: 1.070112
         Iterations: 14
         Function evaluations: 100
         Gradient evaluations: 20


In [54]:
W_spb

array([ 0.00820521, -0.30958003,  0.99053633,  0.22868203])

In [60]:
def minimize_arit(W):
    ypred = W[0] * nn_predicts + W[1] * lgb_predicts + W[2] * xgb_predicts + W[3] * cat_predicts
    return deviation_metric(train_targets, ypred)


W = minimize(minimize_arit, [1.0 / 4] * 4, options={'gtol': 1e-6, 'disp': True}).x
# 1.018536
W 

Optimization terminated successfully.
         Current function value: 1.001246
         Iterations: 12
         Function evaluations: 80
         Gradient evaluations: 16


array([0.07274949, 0.06529732, 0.48726262, 0.28138078])

In [61]:

test_submission = pd.read_csv('dataset/test_submission.csv')
test_submission['per_square_meter_price'] = test_nn_predict * W[0] + test_lgb_predict * W[1] + test_xgb_predict * W[2] + test_cat_predict * W[3]
test_submission['per_square_meter_price'] = test_submission['per_square_meter_price'].apply(lambda x: max(1000.0, x))
test_submission.to_csv('submission.csv', index = False)

In [ ]:
xgb_predicts_spb = xgb_predicts[train.city == 0]
cat_predicts_spb = cat_predicts[train.city == 0]
nn_predicts_spb = nn_predicts[train.city == 0]
lgb_predicts_spb = lgb_predicts[train.city == 0]
train_targets_spb = train_targets[train.city == 0]

test_nn_predict_spb = test_nn_predict[test.city == 0]
test_lgb_predict_spb = test_lgb_predict[test.city == 0]
test_xgb_predict_spb = test_xgb_predict[test.city == 0]
test_cat_predict_spb = test_cat_predict[test.city == 0]

def minimize_arit_spb(W):
    ypred = W[0] * nn_predicts_spb + W[1] * lgb_predicts_spb + W[2] * xgb_predicts_spb + W[3] * cat_predicts_spb
    return deviation_metric(train_targets_spb, ypred)


W_spb = minimize(minimize_arit_spb, [1.0 / 4] * 4, options={'gtol': 1e-6, 'disp': True}).x


In [ ]:
test_submission.loc[test.city == 0, 'per_square_meter_price'] = test_nn_predict_spb * W[0] + test_lgb_predict_spb * W[1] + test_xgb_predict_spb * W[2] + test_cat_predict_spb * W[3]


In [ ]:
test_submission.to_csv('submission_cities.csv', index = False)